In [1]:
# import packages 
from skimage import io, transform
import numpy as np
import open3d as o3d
import copy
import napari
import time
import os

from skimage import morphology
from skimage.measure import label, regionprops, block_reduce
from scipy import stats, ndimage
import matplotlib.pyplot as plt
import sys

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)


from src.preprocess import segmentation_with_optimized_thresh, image_padding, clean_up_segmented_image
from src.registration import image_to_pcd, pcd_to_image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [44]:
t0 = time.time()

# Read the image, only fluorescence channel is required:
#reference_fly_filename = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data/References_and_masks/Reference_abdomen.tif"
#reference_fly_filename = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data/01_raw/output/C1-A0B0_male_1_20190706.tif"
reference_fly_filename = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data/01_raw/output/C1-0_male_1_20190604.tif"
image = io.imread(reference_fly_filename)

# Downscaling:
downscaling =  (12,4,4)
image_downscaled = transform.downscale_local_mean(image, downscaling)[1:-2,1:-2,1:-2]

# Segment:
Thresholded_Image = segmentation_with_optimized_thresh(image_downscaled, fraction_range = [0.03, 0.04])
#viewer = napari.view_image(thresholded)

# Padding:
Thresholded_Image  = image_padding(Thresholded_Image)

# Clean up the segmentation with morphological transformations:
Thresholded_Image = clean_up_segmented_image(Thresholded_Image)

t1 = time.time()

# print running time:

print("Preprocessing of the image took: ", t1-t0, "seconds")

viewer = napari.view_image(Thresholded_Image)

Preprocessing of the image took:  37.63531303405762 seconds


In [45]:
def draw_pcd(pcd):
    pcd_temp = copy.deepcopy(pcd)
    pcd_temp.paint_uniform_color([1, 0.706, 0])
    o3d.visualization.draw_geometries([pcd_temp])

# Create a pcd:
pcd, pcd_values = image_to_pcd(Thresholded_Image)

# Translate and rotate the point cloud:
pcd_mean, pcd_cov = pcd.compute_mean_and_covariance()
_ , pcd_eigv = np.linalg.eig(pcd_cov)

# Transform to a base given by the first eigenvector (a,b,c), (0,0,1) and their vector product: (b,-a,0)

b = pcd_eigv[1,0]
c = pcd_eigv[2,0]
b, c = b/(b**2+c**2)**0.5, c/(b**2+c**2)**0.5

temp = np.asarray([[1.0, 0.0, 0.0], [0.0, b, c], [0.0, c, -b]])
transformation_pcd = np.eye(4)
transformation_pcd[:3, :3] = np.linalg.inv(temp)
    
pcd.translate(-pcd_mean)
pcd.transform(transformation_pcd)

array = np.asarray(pcd.points)

translation = [-min(array[:,0])+10,-min(array[:,1])+20, -min(array[:,2])+10]
pcd.translate(translation)
array = np.asarray(pcd.points)


#draw_pcd(pcd)
#o3d.visualization.draw_geometries([pcd])

def skeletonize_slices(image_3d):
    
    result_1 = np.zeros(image_3d.shape)
    #result_2 = np.zeros(image_3d.shape)
    #result_3 = np.zeros(image_3d.shape)
    
    #for i in range(image_3d.shape[0]):
    #    image = image_3d[i,:,:]
    #    skeleton = morphology.skeletonize(image)
    #    result_1[i,:,:] = skeleton
    
    for i in range(image_3d.shape[1]):
        image = image_3d[:,i,:]
        skeleton = morphology.skeletonize(image)
        result_1[:,i,:] = skeleton
        
    #for i in range(image_3d.shape[2]):
    #    image = image_3d[:,:,i]
    #    skeleton = morphology.skeletonize(image)
    #    result_3[:,:,i] = skeleton
        
        
    return result_1#*result_2*result_3
        

new_reference = pcd_to_image(pcd, pcd_values, Thresholded_Image.shape)

-26.636975148478506 36.363024851521494
-131.18881536350813 119.76843029429409
-66.17375495337274 81.8353746015645
10.0 73.0
20.0 270.9572456578022
10.0 158.00912955493723


In [46]:
#skeletonized = morphology.skeletonize_3d(new_reference)
skeletonized = skeletonize_slices(new_reference)

#viewer = napari.view_image(skeletonized+new_reference)

skeleton_pcd, skeleton_values = image_to_pcd(skeletonized)



uni_down_pcd = skeleton_pcd.uniform_down_sample(every_k_points=3)



cleaned, ind = uni_down_pcd.remove_radius_outlier(nb_points=4, radius=3)
cleaned, ind = cleaned.remove_radius_outlier(nb_points=4, radius=3)

cleaned_values = np.ones(np.asarray(cleaned.points).shape[0])

o3d.visualization.draw_geometries([cleaned])

reference = pcd_to_image(cleaned, cleaned_values, Thresholded_Image.shape)
#reference = morphology.dilation(reference, morphology.ball(3))

#viewer = napari.view_image(new_reference+reference)

In [47]:
skeleton_pcd = cleaned
#skeleton_pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # invalidate existing normals

skeleton_pcd.estimate_normals()
skeleton_pcd.orient_normals_consistent_tangent_plane(k=30)
o3d.visualization.draw_geometries([skeleton_pcd], point_show_normal=True)

poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(skeleton_pcd, depth=5, width=0, scale=1.2, linear_fit=True)[0]

bbox = skeleton_pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

o3d.visualization.draw_geometries([skeleton_pcd, p_mesh_crop ])
pcd = p_mesh_crop.sample_points_uniformly(number_of_points=50000)

p2 = np.concatenate((skeleton_pcd.points, pcd.points), axis=0)

pointSet2 = o3d.geometry.PointCloud()
pointSet2.points = o3d.utility.Vector3dVector(p2)
o3d.visualization.draw_geometries([pointSet2])

pcd = pointSet2
pcd_values = np.ones(np.asarray(pcd.points).shape[0])
reference = pcd_to_image(pcd, pcd_values, Thresholded_Image.shape)
reference = morphology.dilation(reference, morphology.ball(3))

viewer = napari.view_image(new_reference+reference)

In [19]:
Thresholded_Image.shape

(115, 293, 293)

In [5]:
# Save the new reference as tiff file
from tifffile import imsave
output_file_name = "/media/ceolin/Data/Lab Gompel/Projects/Fly_Abdomens/data/References_and_masks/Reference_abdomen_12_4_4.tif"
imsave(output_file_name, skeletonized)